In [1]:
# Path hack so that we can import see library.
import sys, os
sys.path.insert(0, os.path.abspath('..'))

## Defining you own parameter space
This notebook demonstrates how one may define their own Classifier Search Space!

In [2]:
from see.classifiers import Classifier, ClassifierParams

We will use the `Classifier#set_search_space` class method to set our custom search space to be the Classifier Search Space that we can run Genetic Search over. We first define a simple custom space dictionaries that will be passed into `Classifier#set_search_space`.

In [3]:
# First, initialize custom search space dictionaries.

# These two dictionaries represent
# the algorithm space and the
# parameter space respectively.
# The keys of these dictionaries
# are the algorithm names and
# the parameter names respectively.
new_algo_space = dict()
new_param_space = dict()

# The values of the new_algo_space
# dictionary needs to be ClassifierContainer
# Classes (not instances!). Many have already
# been pre-defined
# in the classifiers.py file.

# The values of the parameter space dictionary (i.e. new_param_space)
# needs to be a tuple: (range, description).
# `range` is a list of possible values for the
# the corresponding parameter name.
# `description` is used to document and
# describe the parameter.

In [4]:
# To add a Classifier algorithm to the 
# search space, one first needs to pick or define
# the corresponding classifier container.
# Existing Classifier Containers have
# been defined in classifiers.py.

# First add import and add the container 
# to the algorithm space dictionary.
from see.classifiers import MLPContainer

new_algo_space["MLP Neural Network"] = MLPContainer

# Then, add its parameters to the parameter space dictionary
for param in MLPContainer().paramindexes:
    new_param_space[param] = ClassifierParams.get_param(param)
    
# Right now in order to add a new classifier to the search space
# one would have to define a ClassifierContainer class
# and add its respective parameters and parameter ranges carefully.
# There is not a simpler method 
# to directly add Classifier and custom search space yet.

In [5]:
# Alternatively, one may manually define a search space within
# the Classifier class itself. Look at the Classifier#use_dhahri_space
# and Classifier#use_tutorial_space class methods as an example.

In [6]:
# To set the custom search space as the Classifier space, run:
Classifier.set_search_space(new_algo_space, new_param_space)

In [7]:
# Check the search space by running:
# Initialize Algorithm Space and Workflow
import pprint

algorithm_space = Classifier.algorithmspace
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(algorithm_space)

# Print possible parameters
print('Possible Parameters')
pp.pprint(ClassifierParams.pkeys)
print('\n')
# Print parameter ranges
print('Parameter Ranges')
pp.pprint(ClassifierParams.ranges)
print('\n')
print('Parameter Descriptions')
# Print descriptions of parameters
pp.pprint(ClassifierParams.descriptions)

{'MLP Neural Network': <class 'see.classifiers.MLPContainer'>}
Possible Parameters
['algorithm', 'activation', 'alpha', 'max_iter', 'solver']


Parameter Ranges
{   'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'algorithm': ['MLP Neural Network'],
    'alpha': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1],
    'max_iter': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
    'solver': ['lbfgs', 'sgd', 'adam']}


Parameter Descriptions
{   'activation': 'Activation function for the hidden layer of neural network.',
    'algorithm': 'string code for the algorithm',
    'alpha': 'L2 penalty regularization parameter',
    'max_iter': 'Number of iterations for the algorithm',
    'solver': 'The solver for weight optimization of neural network.'}


Let's test our custom workspace using Genetic Search!

In [8]:
from see.Workflow import workflow
from see.classifier_fitness import ClassifierFitness

# Initialize workflow for Genetic Search
workflow.addalgos([Classifier, ClassifierFitness])
wf = workflow()
print(wf)

<class 'see.Workflow.workflow'> parameters: 
	algorithm = MLP Neural Network
	activation = identity
	alpha = 1e-06
	max_iter = 100
	solver = lbfgs



In [9]:
%%time
from see import GeneticSearch
num_generations = 5

from see.base_classes import pipedata
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_circles
from see.classifier_helpers import helpers

# Create Simple Dataset
circles_ds = pipedata()
circles_ds.name = 'Circles Dataset'
circles_ds.X, circles_ds.y = make_circles(noise=0.2, factor=0.5, random_state=1)

# Preprocess data
circles_ds.X = StandardScaler().fit_transform(circles_ds.X)

# Create pipeline data that can be passed into the classifier pipeline
pipeline_data = helpers.generate_train_test_set(circles_ds.X, circles_ds.y)

my_evolver = GeneticSearch.Evolver(workflow, pipeline_data, pop_size=5)
my_evolver.run(ngen=num_generations, print_raw_data=True)

Initializing a new random population
Generation 0/5 of population size 5
<class 'see.Workflow.workflow'> parameters: 
	algorithm = MLP Neural Network
	activation = identity
	alpha = 1e-06
	max_iter = 800
	solver = adam

Time: 0.074 s
fitness=0.6

<class 'see.Workflow.workflow'> parameters: 
	algorithm = MLP Neural Network
	activation = relu
	alpha = 1e-05
	max_iter = 800
	solver = adam



/home/davin/Documents/GitHub/see-segment/see-clasify/envs/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  warnings.warn(


Time: 0.628 s
fitness=0.125

<class 'see.Workflow.workflow'> parameters: 
	algorithm = MLP Neural Network
	activation = relu
	alpha = 1e-06
	max_iter = 1000
	solver = lbfgs

Time: 0.048 s
fitness=0.125

<class 'see.Workflow.workflow'> parameters: 
	algorithm = MLP Neural Network
	activation = tanh
	alpha = 0.0001
	max_iter = 100
	solver = lbfgs

Time: 0.063 s
fitness=0.19999999999999996

<class 'see.Workflow.workflow'> parameters: 
	algorithm = MLP Neural Network
	activation = relu
	alpha = 0.001
	max_iter = 500
	solver = lbfgs



/home/davin/Documents/GitHub/see-segment/see-clasify/envs/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Time: 0.532 s
fitness=0.125

#BEST [0.125,  ['MLP Neural Network', 'relu', 0.001, 500, 'lbfgs']]
# GEN HOF_index fitness ind|0;0;0.125;['MLP Neural Network', 'relu', 0.001, 500, 'lbfgs']
# GEN HOF_index fitness ind|0;1;0.125;['MLP Neural Network', 'relu', 1e-06, 1000, 'lbfgs']
# GEN HOF_index fitness ind|0;2;0.125;['MLP Neural Network', 'relu', 1e-05, 800, 'adam']
# GEN HOF_index fitness ind|0;3;0.19999999999999996;['MLP Neural Network', 'tanh', 0.0001, 100, 'lbfgs']
# GEN HOF_index fitness ind|0;4;0.6;['MLP Neural Network', 'identity', 1e-06, 800, 'adam']
# GEN population_index fitness ind|0;0;0.6;['MLP Neural Network', 'identity', 1e-06, 800, 'adam']
# GEN population_index fitness ind|0;1;0.125;['MLP Neural Network', 'relu', 1e-05, 800, 'adam']
# GEN population_index fitness ind|0;2;0.125;['MLP Neural Network', 'relu', 1e-06, 1000, 'lbfgs']
# GEN population_index fitness ind|0;3;0.19999999999999996;['MLP Neural Network', 'tanh', 0.0001, 100, 'lbfgs']
# GEN population_index fitness in

...

Time: 0.537 s
fitness=0.09999999999999998

<class 'see.Workflow.workflow'> parameters: 
	algorithm = MLP Neural Network
	activation = relu
	alpha = 0.001
	max_iter = 600
	solver = adam



/home/davin/Documents/GitHub/see-segment/see-clasify/envs/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(


Time: 0.536 s
fitness=0.125

<class 'see.Workflow.workflow'> parameters: 
	algorithm = MLP Neural Network
	activation = relu
	alpha = 0.001
	max_iter = 500
	solver = adam

Time: 0.479 s
fitness=0.09999999999999998

#BEST [0.09999999999999998,  ['MLP Neural Network', 'relu', 0.001, 600, 'adam']]
# GEN HOF_index fitness ind|5;0;0.09999999999999998;['MLP Neural Network', 'relu', 0.001, 600, 'adam']
# GEN HOF_index fitness ind|5;1;0.09999999999999998;['MLP Neural Network', 'relu', 0.001, 500, 'adam']
# GEN HOF_index fitness ind|5;2;0.125;['MLP Neural Network', 'tanh', 0.001, 500, 'adam']
# GEN HOF_index fitness ind|5;3;0.125;['MLP Neural Network', 'tanh', 0.001, 600, 'adam']
# GEN HOF_index fitness ind|5;4;0.125;['MLP Neural Network', 'relu', 1e-06, 500, 'lbfgs']
# GEN HOF_index fitness ind|5;5;0.22499999999999998;['MLP Neural Network', 'relu', 0.001, 500, 'lbfgs']
# GEN HOF_index fitness ind|5;6;0.125;['MLP Neural Network', 'relu', 1e-06, 1000, 'lbfgs']
# GEN HOF_index fitness ind|5;7;0.1

/home/davin/Documents/GitHub/see-segment/see-clasify/envs/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[['MLP Neural Network', 'relu', 0.0001, 200, 'sgd'],
 ['MLP Neural Network', 'relu', 1e-05, 1000, 'lbfgs'],
 ['MLP Neural Network', 'relu', 0.001, 600, 'adam'],
 ['MLP Neural Network', 'relu', 0.001, 600, 'adam'],
 ['MLP Neural Network', 'relu', 0.001, 500, 'adam']]

## Modifying the search space
This section demonstrates how one may modify the own Classifier Search Space!

In [10]:
# Imports
from see.classifiers import Classifier, ClassifierParams

In [11]:
# To see the range and description of one parameter (such as 'algorithm')
ClassifierParams.get_param('algorithm')

(['MLP Neural Network'], 'string code for the algorithm')

In [12]:
# To see the parameters of each algorithm
for clf in Classifier.algorithmspace:
    params = (Classifier.algorithmspace[clf]().paramindexes)
    # To see the ranges
    print('Algorithm: ', clf)
    print('-------------------------------')
    for param_name in params:
        print(param_name)
        ranges, description = ClassifierParams.get_param(param_name)
        print('description: ', description)
        print('range: ', ranges)
        print('\n')
    print('\n')

Algorithm:  MLP Neural Network
-------------------------------
activation
description:  Activation function for the hidden layer of neural network.
range:  ['identity', 'logistic', 'tanh', 'relu']


alpha
description:  L2 penalty regularization parameter
range:  [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1]


max_iter
description:  Number of iterations for the algorithm
range:  [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]


solver
description:  The solver for weight optimization of neural network.
range:  ['lbfgs', 'sgd', 'adam']






In [13]:
# To change the range
ClassifierParams.set_param_ranges('activation', ['o'])
# To change the description
ClassifierParams.set_param_description('activation', 'bad description')

In [14]:
# Check the new range and description
ranges, description = ClassifierParams.get_param('activation')
print('ranges: ', ranges)
print('description: ', description)

ranges:  ['o']
description:  bad description


In [15]:
# To add a parameter
ClassifierParams.add('l', [0, 1], 'random letter')

In [16]:
# To see check the parameter space again for the new parameter
for param_name in ClassifierParams.pkeys:
    print('Parameter name: ', param_name)
    ranges, description = ClassifierParams.get_param(param_name)
    print('description: ', description)
    print('ranges: ', ranges)
    print('\n')

Parameter name:  algorithm
description:  string code for the algorithm
ranges:  ['MLP Neural Network']


Parameter name:  activation
description:  bad description
ranges:  ['o']


Parameter name:  alpha
description:  L2 penalty regularization parameter
ranges:  [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1]


Parameter name:  max_iter
description:  Number of iterations for the algorithm
ranges:  [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]


Parameter name:  solver
description:  The solver for weight optimization of neural network.
ranges:  ['lbfgs', 'sgd', 'adam']


Parameter name:  l
description:  random letter
ranges:  [0, 1]


